In [93]:
from os import listdir
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
import keras.layers as L
from keras.models import Model
from skimage.transform import resize
from keras.callbacks import ModelCheckpoint
from skimage import util
from keras.models import load_model

%matplotlib inline

In [31]:
names = sorted(listdir('public_data/00_input/train/images/'))

points = pd.read_csv('public_data/00_gt/gt.csv')
ar_points = np.array(points)[:, 1:]

images = []
noise_images = []
test_images = []
test_points = []
indeces_delete = []
size = (100, 100, 3)
for i, name in enumerate(names):
    image = imread('public_data/00_input/train/images/' + name)
    k_height, k_width = size[0] / image.shape[0], size[1] / image.shape[1]
    
    ar_points[i][::2] = ar_points[i][::2] * k_height
    ar_points[i][1::2] = ar_points[i][1::2] * k_width
    image = resize(image, output_shape=(100, 100, 3))
    if i % 60 == 0:
        test_images.append(image)
        test_points.append(ar_points[i])
        indeces_delete.append(i)
    else:
        images.append(image)
        noise_images.append(util.random_noise(image))
    
ar_points = np.delete(ar_points, indeces_delete, axis=0)
images = np.vstack((images, noise_images))
ar_points = np.vstack((ar_points, ar_points))

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [33]:
test_images = np.array(test_images)
test_points = np.array(test_points)
test_images.shape, test_points.shape

((100, 100, 100, 3), (100, 28))

In [34]:
def show_image(image, point):
    plt.imshow(image)
    for i in range(len(point) // 2):
        plt.scatter(point[2*i], point[2*i + 1], c='r')
    plt.show()

In [35]:
# horizontal flip
perm = np.array([6, 7, 4, 5, 2, 3, 0, 1, 18, 19, 16, 17, 14, 15, 12, 13, 10, 11, 8, 9, 20, 21, 26, 27, 24, 25, 22, 23])

flip_images = images[:, :,::-1]
flip_points = ar_points[:,perm]
flip_points[:,::2] = 100 - flip_points[:,::2]

ar_points = np.vstack((ar_points, flip_points))
images = np.vstack((images, flip_images))

In [8]:
# def random_rotation(image_array, points):
#     # pick a random degree of rotation between 25% on the left and 25% on the right
#     random_degree = random.uniform(-25, 25)
#     return skimage.transform.rotate(image_array, random_degree)

# def random_noise(image_array):
#     # add random noise to the image
#     return sk.util.random_noise(image_array)

# def horizontal_flip(image_array):
#     # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
#     return image_array[:, ::-1]

In [36]:
# def preprocessing(images, points):
#     ar_points = np.array(points)[:, 1:]
#     size = (100, 100, 3)
#     resize_images = images.copy()
#     for i, image in enumerate(images):
#         k_height, k_width = size[0] / image.shape[0], size[1] / image.shape[1]
#         resize_images[i] = resize(image, output_shape=size)
#         ar_points[i][::2] = ar_points[i][::2] * k_height
#         ar_points[i][1::2] = ar_points[i][1::2] * k_width
#         #data_augumentation
        
#     resize_images = np.array(resize_images)
# #     mean_images = np.mean(images, axis=0)
# #     images -= mean_images
    
#     return resize_images, ar_points



In [ ]:
size = (100, 100, 3)
input_images = L.Input(shape=size)

conv1 = L.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(L.BatchNormalization() (input_images))
pool1 = L.MaxPool2D(pool_size=(2, 2), strides=2)(conv1)

conv2 = L.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(pool1)
pool2 = L.MaxPool2D(pool_size=(2, 2), strides=2)(conv2)

conv3 = L.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(pool2)
pool3 = L.MaxPool2D(pool_size=(2, 2), strides=2)(conv3)

conv4 = L.Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(pool3)
pool4 = L.MaxPool2D(pool_size=(2, 2), strides=2)(conv4)

flatten = L.Flatten()(pool4)
dense1 = L.Dense(units=256, activation='relu')(flatten)
dropout = L.Dropout(0.1)(dense1)
result = L.Dense(units=28)(dropout)

model = Model(inputs=input_images, outputs=result)

checkpoint = ModelCheckpoint('facepoints_model.hdf5', save_best_only=True)

model.compile('adam', loss='mse')

model.fit(x=images, y=ar_points.astype(np.float32), epochs=50, batch_size=32, callbacks=[checkpoint], validation_data=(test_images, test_points))


Train on 23600 samples, validate on 100 samples
Epoch 1/50
23600/23600 [==============================] - 20s 849us/step - loss: 82.0268 - val_loss: 18.9901
Epoch 2/50
23600/23600 [==============================] - 18s 747us/step - loss: 38.3872 - val_loss: 18.2117
Epoch 3/50
23600/23600 [==============================] - 17s 731us/step - loss: 27.8446 - val_loss: 12.2764
Epoch 4/50
23600/23600 [==============================] - 17s 736us/step - loss: 22.9302 - val_loss: 9.6141
Epoch 5/50
23600/23600 [==============================] - 18s 774us/step - loss: 20.2140 - val_loss: 9.8952
Epoch 6/50
23600/23600 [==============================] - 19s 791us/step - loss: 18.5342 - val_loss: 6.8467
Epoch 7/50
23600/23600 [==============================] - 18s 783us/step - loss: 16.2258 - val_loss: 7.0744
Epoch 8/50
23600/23600 [==============================] - 18s 758us/step - loss: 14.5028 - val_loss: 9.4865
Epoch 9/50
23600/23600 [==============================] - 17s 725us/step - loss: 13.9

In [12]:
from sklearn.metrics import mean_squared_error
pred = model.predict(test_images)
mean_squared_error(pred, test_points)

4.830527064689677

In [13]:
mean_squared_error(model.predict(images), ar_points)

3.991602609227147

In [15]:
from keras.models import load_model

model_weights = load_model('facepoints_model.hdf5')
mean_squared_error(model_weights.predict(test_images), test_points)

4.830527064689677

In [4]:
model = load_model('facepoints_model.hdf5')

In [24]:
from os.path import join
from os import listdir
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
import keras
import keras.layers as L
from keras.models import Model
from skimage.transform import resize
from keras.callbacks import ModelCheckpoint
from skimage import util
from keras.models import load_model


def train_detector(train_gt, train_img_dir, fast_train=True):
    size = (100, 100, 3)
    
    names = sorted(listdir(train_img_dir))
    names=names[:250]
#     points = pd.read_csv(train_gt)
#     ar_points = (np.array(points)[:, 1:]).astype(np.float32)
    ar_points = np.ones((len(names), 28))

    images = []
    noise_images = []

    for i, name in enumerate(names):
        image = imread(join(train_img_dir, name))
        k_height, k_width = size[0] / image.shape[0], size[1] / image.shape[1]
        ar_points[i] = train_gt[name]
        ar_points[i][::2] = ar_points[i][::2] * k_height
        ar_points[i][1::2] = ar_points[i][1::2] * k_width
        image = resize(image, output_shape=(100, 100, 3))
        images.append(image)
        noise_images.append(util.random_noise(image))

    images = np.vstack((images, noise_images))
    ar_points = np.vstack((ar_points, ar_points))
    
    perm = np.array([6, 7, 4, 5, 2, 3, 0, 1, 18, 19, 16, 17, 14, 15, 12, 13, 10, 11, 8, 9, 20, 21, 26, 27, 24, 25, 22, 23])

    flip_images = images[:, :,::-1]
    flip_points = ar_points[:,perm]
    flip_points[:,::2] = 100 - flip_points[:,::2]

    ar_points = np.vstack((ar_points, flip_points))
    images = np.vstack((images, flip_images))
    
    
    input_images = L.Input(shape=size)

    conv1 = L.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(L.BatchNormalization() (input_images))
    pool1 = L.MaxPool2D(pool_size=(2, 2), strides=2)(conv1)

    conv2 = L.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(pool1)
    pool2 = L.MaxPool2D(pool_size=(2, 2), strides=2)(conv2)

    conv3 = L.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(pool2)
    pool3 = L.MaxPool2D(pool_size=(2, 2), strides=2)(conv3)

    conv4 = L.Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(pool3)
    pool4 = L.MaxPool2D(pool_size=(2, 2), strides=2)(conv4)

    flatten = L.Flatten()(pool4)
    dense1 = L.Dense(units=256, activation='relu')(flatten)
    dropout = L.Dropout(0.1)(dense1)
    result = L.Dense(units=28)(dropout)

    model = Model(inputs=input_images, outputs=result)
#     checkpoint = ModelCheckpoint('facepoints_model.hdf5', save_best_only=True)
    model.compile('adam', loss='mse')
    
    if fast_train:
        epochs = 1
    else:
        epochs = 40
        
    model.fit(x=images, y=ar_points.astype(np.float32), epochs=epochs, batch_size=32)

    return model

def detect(model, directory) :
    names = np.array(sorted(listdir(directory)))
    size = (100, 100, 3)
    test_images = []
    test_flipped_images = []
    batch_size = 500
    batch_count = (len(names) + batch_size - 1) // batch_size
    all_prediction = []
    shapes = []
    for i in range(batch_count):
        test_images = []
        end = min(len(names), (i + 1) * batch_size)
        for name in names[i * batch_size: end]:
            image = imread(join(directory, name))
            shapes.append(image.shape)
            image = resize(image, output_shape=(100, 100, 3))
            test_images.append(image)
            
        test_images = np.array(test_images)
        test_flipped_images = test_images[:, :,::-1]
        prediction = model.predict(test_images)
        prediction_flipped = model.predict(test_flipped_images)
        perm = np.array([6, 7, 4, 5, 2, 3, 0, 1, 18, 19, 16, 17, 14, 15, 12, 13, 10, 11, 8, 9, 20, 21, 26, 27, 24, 25, 22, 23])
        prediction_flipped = prediction_flipped[:,perm]
        prediction_flipped[:,::2] = 100 - prediction_flipped[:,::2]
        
        prediction = (prediction + prediction_flipped) / 2
        
        if i == 0:
            all_prediction = prediction
        else:
            all_prediction = np.vstack((all_prediction, prediction))  
    
    for i, shape in enumerate(shapes):
        k_height, k_width = shape[0] / size[0] , shape[1] / size[1]
        all_prediction[i][::2]  = all_prediction[i][::2] * k_height
        all_prediction[i][1::2]  = all_prediction[i][1::2] * k_width
        
    return dict(zip(names, all_prediction))
        
        

In [25]:
code_dir = ''
test_dir = 'public_data/00_input/test/'
model = load_model(join(code_dir, 'facepoints_model.hdf5'))
test_img_dir = join(test_dir, 'images')
detected_points = detect(model, test_img_dir)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [26]:
detected_points

{'00000.jpg': array([ 61.943474,  50.79786 , 126.31335 ,  56.737503, 171.69887 ,
         64.79271 , 203.1297  ,  62.50664 ,  70.201004,  73.28152 ,
         91.52951 ,  74.57629 , 111.30738 ,  78.44071 , 162.66963 ,
         83.94202 , 176.92632 ,  83.297226, 191.35838 ,  85.24741 ,
        162.24323 , 144.25427 ,  93.47738 , 175.48839 , 138.77975 ,
        183.16586 , 163.01831 , 180.30713 ], dtype=float32),
 '00001.jpg': array([22.21503 , 19.116137, 38.96279 , 19.4134  , 58.7097  , 20.697718,
        79.57887 , 23.926044, 25.600168, 27.540588, 32.13364 , 27.561441,
        38.27535 , 28.450947, 60.50218 , 30.252995, 67.52503 , 30.373442,
        74.941635, 31.484083, 43.798553, 48.853264, 31.390572, 63.135437,
        44.971436, 65.813515, 61.53899 , 65.50028 ], dtype=float32),
 '00002.jpg': array([ 21.75239 , 115.243454,  35.45749 ,  82.54814 ,  45.526497,
         50.20964 ,  55.343323,  20.069302,  39.33114 , 114.616104,
         43.10532 , 101.21036 ,  48.560116,  90.25726 ,  61

In [20]:
import pandas as pd
points = pd.read_csv('public_data/00_gt/gt.csv')

In [27]:
# points

In [28]:
def compute_metric(detected, gt, img_shapes):
    res = 0.0
    for filename, coords in detected.items():
        n_rows, n_cols = img_shapes[filename]
        diff = (coords - gt[filename])
        diff[::2] /= n_cols
        diff[1::2] /= n_rows
        diff *= 100
        res += (diff ** 2).mean()
    return res / len(detected.keys())

def read_img_shapes(gt_dir):
    img_shapes = {}
    with open(join(gt_dir, 'img_shapes.csv')) as fhandle:
        next(fhandle)
        for line in fhandle:
            parts = line.rstrip('\n').split(',')
            filename = parts[0]
            n_rows, n_cols = map(int, parts[1:])
            img_shapes[filename] = (n_rows, n_cols)
    return img_shapes

# detected = read_csv(join(output_dir, 'output.csv'))
gt_dir = 'public_data/00_gt'
gt = read_csv(join(gt_dir, 'gt.csv'))
img_shapes = read_img_shapes(gt_dir)
error = compute_metric(detected_points, gt, img_shapes)

In [29]:
error

3.512130033182515

In [6]:
def read_csv(filename):
    res = {}
    with open(filename) as fhandle:
        next(fhandle)
        for line in fhandle:
            parts = line.rstrip('\n').split(',')
            coords = np.array([float(x) for x in parts[1:]], dtype='float64')
            res[parts[0]] = coords
    return res

train_dir = 'public_data/00_input/train'
train_gt = read_csv(join(train_dir, 'gt.csv'))

In [12]:
%%time
m = train_detector(train_gt, 'public_data/00_input/train/images', fast_train=True)

Epoch 1/1
1000/1000 [==============================] - 19s 19ms/step - loss: 560.8406
CPU times: user 1min 1s, sys: 9.67 s, total: 1min 11s
Wall time: 25.7 s


Без аугументации на тесте

In [102]:
from sklearn.metrics import mean_squared_error

pred_data_train = detect(model, 'public_data/00_input/test/images/')

mean_squared_error(np.array(pred_data_train.iloc[:,1:]), ar_points)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


3.9509453819339675

С аугументацией на тесте

In [100]:
from sklearn.metrics import mean_squared_error

pred_data_train = detect(model, 'public_data/00_input/test/images/')

mean_squared_error(np.array(pred_data_train.iloc[:,1:]), ar_points)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


3.5131273849398488

In [88]:
directory = 'public_data/00_input/train/images/'
names = sorted(listdir(directory))

points = pd.read_csv('public_data/00_gt/gt.csv')
ar_points = np.array(points)[:, 1:]

t_images = []

size = (100, 100, 3)
for i, name in enumerate(names):
    image = imread(directory + name)
    k_height, k_width = size[0] / image.shape[0], size[1] / image.shape[1]
    
    ar_points[i][::2] = ar_points[i][::2] * k_height
    ar_points[i][1::2] = ar_points[i][1::2] * k_width
    image = resize(image, output_shape=(100, 100, 3))

    t_images.append(image)